In [ ]:
! pip install tensorflow-text


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
df = pd.read_csv("spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [ ]:
df['Category'].value_counts()

,count
Category,
ham,4825
spam,747


In [ ]:
# we will use down sampling here to handle the imbalance dataset

In [ ]:
747/4825

0.15481865284974095

In [ ]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [ ]:
df_ham = df[df['Category'] == 'ham']
df_ham.shape

(4825, 2)

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [ ]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced.shape

(1494, 2)

In [ ]:
df_balanced.describe()

,Category,Message
count,1494,1494
unique,2,1375
top,spam,"Sorry, I'll call later"
freq,747,9


In [ ]:
df_balanced['Category'].value_counts()

,count
Category,
spam,747
ham,747


In [ ]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message,spam
1196,spam,You have 1 new voicemail. Please call 08719181503,1
4793,ham,Gd luck 4 ur exams :-),0
5479,ham,Really do hope the work doesnt get stressful. ...,0
4299,spam,WELL DONE! Your 4* Costa Del Sol Holiday or £5...,1
2246,ham,How did you find out in a way that didn't incl...,0


In [ ]:
! pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify=df_balanced['spam'])

In [ ]:
X_train.head(5)

,Message
1595,Never blame a day in ur life. Good days give u...
3780,"Claim a 200 shopping spree, just call 08717895..."
4930,Got what it takes 2 take part in the WRC Rally...
914,Ok lor but not too early. Me still having proj...
3167,HOT LIVE FANTASIES call now 08707509020 Just 2...


In [ ]:
preprocessing_url = 'https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3'
encoder_url = 'https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-12-h-768-a-12/2'

In [ ]:
bert_preprocess_model = hub.KerasLayer(preprocessing_url)
bert_encoder_model = hub.KerasLayer(encoder_url)

In [ ]:
def get_sentence_embedding(sentences):
  pre_processed_text = bert_preprocess_model(sentences)
  return bert_encoder_model(pre_processed_text)['pooled_output']

In [ ]:
get_sentence_embedding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[ 0.07327899, -0.24222296,  0.7754133 , ..., -0.77640074,
         0.37399286,  0.9943151 ],
       [-0.14638081, -0.47830737, -0.60078675, ..., -0.9089972 ,
         0.4568342 ,  0.98879844]], dtype=float32)>

In [ ]:
e = get_sentence_embedding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]], [e[1]])

array([[0.71183336]], dtype=float32)

In [ ]:
cosine_similarity([e[0]],[e[3]])

array([[0.03427682]], dtype=float32)

In [ ]:
# bert_preprocess_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)

In [ ]:
# import tensorflow._api.v2.compat.v1 as tf

# tf.disable_v2_behavior()

In [ ]:
# ! pip install transformers

In [ ]:
# from transformers import BertTokenizer, TFBertModel

# # Load the BERT tokenizer and BERT model from Hugging Face's transformers library
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Function to preprocess the text input
# # def preprocess_text(texts):
#     return tokenizer(texts, padding=True, truncation=True, return_tensors='tf')


In [ ]:
# from tensorflow.python.framework.ops import disable_eager_execution

# disable_eager_execution()

In [ ]:
! pip install keras-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.2/572.2 kB 25.8 MB/s eta 0:00:00


In [ ]:
# Bert layers
# tf.compat.v1.disable_eager_execution()
import keras_nlp
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased",trainable=True)
encoder_inputs = preprocessor(text_input)
encoder = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")
outputs = encoder(encoder_inputs)


# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

100%|██████████| 141/141 [00:00<00:00, 275kB/s]


100%|██████████| 547/547 [00:00<00:00, 1.01MB/s]


100%|██████████| 226k/226k [00:00<00:00, 292kB/s]


100%|██████████| 510/510 [00:00<00:00, 1.04MB/s]


100%|██████████| 418M/418M [00:31<00:00, 13.9MB/s]


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)         │ (None)                 │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_preprocessor         │ [(None, 512), (None,   │              0 │ text[0][0]             │
│ (BertPreprocessor)        │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 768), (None,   │    109,482,240 │ bert_preprocessor[0][… │
│ (BertBackbone)            │ 512, 768)]             │                │ bert_preprocessor[0][… │
│                           │                        │                │ bert_preprocessor[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 768)            │              0 │ bert_backbone[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │            769 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 109,483,009 (417.64 MB)

 Trainable params: 109,483,009 (417.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
len(X_train)

1120

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
! pip install pydot
! pip install pydotplus
! sudo apt-get install graphviz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node data defined at (most recent call last):
<stack traces unavailable>
Detected at node data defined at (most recent call last):
<stack traces unavailable>
Detected unsupported operations when trying to compile graph __inference_one_step_on_data_100185[] on XLA_GPU_JIT: _Arg (No registered '_Arg' OpKernel for XLA_GPU_JIT devices compatible with node {{node data}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_STRING, _output_shapes=[[32,1]], _user_specified_name="data", index=0){{node data}}
The op is created at: 
File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
File "/usr/local/lib/python3.10/dist-packages/ipykernel/